In [20]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [11]:
Datadirectory = "C:/Users/bened/Downloads/Real-time Drowsiness Detection/Train_Dataset" ##training dataset
Classes = ["Closed_Eyes","Open_Eyes"] ##List of Classes

In [34]:
training_Data=[]

def create_training_Data():
    for category in Classes:
        path =os.path.join(Datadirectory, category)
        class_num = Classes.index(category) ## 0,1
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                backtorgb = cv2.cvtColor(img_array,cv2.COLOR_GRAY2RGB) 
                new_array = cv2.resize(backtorgb, (img_size,img_size))
                training_Data.append([new_array,class_num])
            except Exception as e:
                pass

In [35]:
create_training_Data()

In [36]:
print(len(training_Data))

1691


In [37]:
import random

random.shuffle(training_Data) #shuffling to avoid overfitting

In [38]:
img_size =224

X = []
y = []

for features,label in training_Data:
    X.append(features) #data
    y.append(label)

X = np.array(X).reshape(-1 , img_size, img_size, 3) #3-number of channels

In [39]:
X.shape # 1448 number of images in dataset

(1691, 224, 224, 3)

In [40]:
#normalize the data
X= X/255.0; ## we are normalizing it  
#better to use sk -learn

In [41]:
Y = np.array(y) #saving labels to np array

In [42]:
import pickle

# Assuming X and y are numpy arrays that you want to save
# X and y should be defined and initialized before this code snippet

# Save X using pickle
with open("X.pickle", "wb") as pickle_out:
    pickle.dump(X, pickle_out)

# Save y using pickle
with open("y.pickle", "wb") as pickle_out:
    pickle.dump(y, pickle_out)

""""
Persistence: Pickling allows you to save complex Python objects 
to disk, databases, or network storage, 
preserving their structure and state.
This is useful for saving program state, 
caching, or sharing data between different applications.
"""


'"\nPersistence: Pickling allows you to save complex Python objects \nto disk, databases, or network storage, \npreserving their structure and state.\nThis is useful for saving program state, \ncaching, or sharing data between different applications.\n'

In [43]:
pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

#normally things get saved in ram after restarting we have to run from start
#to load the variable functionality of picker 

In [44]:
import tensorflow as tf
from tensorflow import keras # tesorflow contain many predefined deeplearning algorithms
from tensorflow.keras import layers

In [45]:
model = tf.keras.applications.mobilenet.MobileNet()

In [46]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32

 )                                                               
                                                                 
 conv_dw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_dw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_8 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D  (None, 14, 14, 512)       4608      
 )        

In [47]:
base_input = model.layers[0].input ##input

In [48]:
base_output = model.layers[-4].output #output - dropout layer

In [49]:
Flat_layer= layers.Flatten()(base_output)
final_output = layers.Dense(1)(Flat_layer) ## one node (1/0)
final_output = layers.Activation('sigmoid')(final_output)

In [50]:
new_model = keras.Model(inputs =base_input, outputs = final_output)

In [51]:
new_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128 

 )                                                               
                                                                 
 conv_dw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_dw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_pw_8 (Conv2D)          (None, 14, 14, 512)       262144    
                                                                 
 conv_pw_8_bn (BatchNormali  (None, 14, 14, 512)       2048      
 zation)                                                         
                                                                 
 conv_pw_8_relu (ReLU)       (None, 14, 14, 512)       0         
                                                                 
 conv_dw_9 (DepthwiseConv2D  (None, 14, 14, 512)       4608      
 )        

In [52]:
new_model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [44]:
new_model.fit(X,Y,epochs =10,validation_split =0.1) ##trainig

Epoch 1/10
48/48 [==============================] - 108s 2s/step - loss: 0.1051 - accuracy: 0.9704 - val_loss: 7.3267 - val_accuracy: 0.6118
Epoch 2/10
48/48 [==============================] - 104s 2s/step - loss: 0.0395 - accuracy: 0.9908 - val_loss: 2.9340 - val_accuracy: 0.7412
Epoch 3/10
48/48 [==============================] - 108s 2s/step - loss: 0.0051 - accuracy: 0.9980 - val_loss: 1.7218 - val_accuracy: 0.7353
Epoch 4/10
48/48 [==============================] - 103s 2s/step - loss: 0.0109 - accuracy: 0.9967 - val_loss: 0.4199 - val_accuracy: 0.9118
Epoch 5/10
48/48 [==============================] - 120s 2s/step - loss: 3.6431e-04 - accuracy: 1.0000 - val_loss: 0.2803 - val_accuracy: 0.9235
Epoch 6/10
48/48 [==============================] - 137s 3s/step - loss: 9.0992e-04 - accuracy: 1.0000 - val_loss: 0.1420 - val_accuracy: 0.9765
Epoch 7/10
48/48 [==============================] - 175s 4s/step - loss: 8.2223e-05 - accuracy: 1.0000 - val_loss: 0.0723 - val_accuracy: 0.9824
E

In [53]:
new_model.save('epochs10.h5')

In [21]:
new_model = tf.keras.models.load_model('TRAIN_my_model.h5')

In [24]:
import cv2
import numpy as np

path = "haarcascade_frontalface_alt.xml"
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_alt.xml')

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, frame = cap.read()
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, 1.1, 4)
    
    for x, y, w, h in eyes:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        eyess = eye_cascade.detectMultiScale(roi_gray)
        
        if len(eyess) == 0:
            print("eyes are not detected")
        else:
            for (ex, ey, ew, eh) in eyess:
                eyes_roi = roi_color[ey: ey+eh, ex:ex+ew]
                
    final_image = cv2.resize(eyes_roi, (224, 224))
    final_image = np.expand_dims(final_image, axis=0)
    final_image = final_image / 255.0

    Predictions = new_model.predict(final_image)
    
    prediction_value = Predictions[0][0]  # Fixed variable name here
    if prediction_value > 0.5:
        status = "Open Eyes"
    else:
        status = "Closed Eyes"
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray, 1.1, 4)
    
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, status, (50, 50), font, 3, (0, 0, 255), 2, cv2.LINE_4)
    cv2.imshow('Drowsiness Detection Tutorial', frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 76ms/step
eyes are not detected
1/1 [==============================] - 0s 77ms/step
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 81ms/step
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 47ms/step
eyes are not detected
1/1 [==============================] - 0s 42ms/step
eyes are not detected
1/1 [==============================] - 0s 45ms/step
eyes are not detected
eyes are not detected
1/1 [==============================] - 0s 89ms/step
eyes are not detected
1/1 [==============================] - 0s 105ms/step
eyes are not detected
1/1 [==============================] - 0s 74ms/step
eyes are not detected
1/1 [==============================] - 0s 61ms/step
eyes are not detected
1/1 [==============================] - 0s 83ms/step
eyes are not detected
1/1 [==============================] - 0s 68ms/step
eyes are not detected
1/1 [==============================] - 0s 65m

KeyboardInterrupt: 